In [ ]:
import tensorflow as tf
import numpy as np
from keras import layers, models
from sklearn.metrics import classification_report

In [ ]:
image_size = (256, 256) 
batch_size = 32 
train_dir = 'ML Project/../input/brain-tumor-classification-mri/Training' 
test_dir = 'ML Project/../input/brain-tumor-classification-mri/Testing' 


data_augmentation = tf.keras.Sequential(
[
tf.keras.layers.RandomRotation(0.25), 
]
)


train_dataset = tf.keras.utils.image_dataset_from_directory(
train_dir,
labels='inferred', # detect classes from input folder
label_mode='categorical', # One-hot encoding
image_size=image_size, # Resize images
interpolation='nearest', # Resize method
batch_size=batch_size,
shuffle=True # Shuffle the data
)

#Apply data_augmentation to images
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
num_parallel_calls=tf.data.AUTOTUNE)


test_dataset = tf.keras.utils.image_dataset_from_directory(
test_dir,
labels='inferred',
label_mode='categorical',
image_size=image_size,
interpolation='nearest',
batch_size=batch_size,
shuffle=False 
)

# Optimize loading the dataset
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)



Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [ ]:
#Arrays to store training data (numerical values)
X_train_list = []
y_train_list = []


for image_batch, label_batch in train_dataset:
  X_train_list.append(image_batch.numpy()) 
  y_train_list.append(label_batch.numpy()) 

#Conver to np arrays
X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)


#Same as for testing data
X_test_list = []
y_test_list = []


for image_batch, label_batch in test_dataset:
  X_test_list.append(image_batch.numpy())
  y_test_list.append(label_batch.numpy())


X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)


print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)




X_train shape: (2870, 256, 256, 3)
y_train shape: (2870, 4)
X_test shape: (394, 256, 256, 3)
y_test shape: (394, 4)


In [14]:
#Build the CNN model
model = models.Sequential([
    layers.Input(shape = X_train.shape[1:]),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),

    layers.Dropout(0.25),
    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(4, activation='softmax'),
])

#Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,811,588 (25.98 MB)

 Trainable params: 6,811,588 (25.98 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(X_train, y_train, epochs=30, batch_size=batch_size, validation_split=0.2)

Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 44s 564ms/step - accuracy: 0.3343 - loss: 20.5907 - val_accuracy: 0.5035 - val_loss: 1.1545
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 535ms/step - accuracy: 0.5048 - loss: 1.1261 - val_accuracy: 0.5749 - val_loss: 1.0275
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 546ms/step - accuracy: 0.5472 - loss: 1.0413 - val_accuracy: 0.6237 - val_loss: 0.9587
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 40s 558ms/step - accuracy: 0.6307 - loss: 0.9209 - val_accuracy: 0.6446 - val_loss: 0.8858
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 548ms/step - accuracy: 0.6518 - loss: 0.8194 - val_accuracy: 0.6794 - val_loss: 0.8556
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 536ms/step - accuracy: 0.7044 - loss: 0.7393 - val_accuracy: 0.7073 - val_loss: 0.7938
Epoch 7/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 39s 546ms/step - accuracy: 0.7254 - loss: 0.7076 - val_accuracy: 0.7160 - val_loss: 0.7876
Epoch 8/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 41s 576ms/step - accuracy: 0.7550 - loss: 0.6091 - val_acc

In [22]:
#Perfrom prediction
y_pred = model.predict(X_test)

#Convert the highest probability to corresponding class label (index)
y_test_label = np.argmax(y_test, axis=1)
y_pred_label = np.argmax(y_pred, axis=1)

#Compare to true labels
print(classification_report(y_test_label, y_pred_label, target_names=['glioma', 'meningioma', 'no_tumor', 'pituitary']))

#Report accuracy and 
loss, accuracy = model.evaluate(X_test, y_test, verbose = 2)
print("Accuracy: ", accuracy)
print("Loss: ", loss)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step
              precision    recall  f1-score   support

      glioma       0.59      0.13      0.21       100
  meningioma       0.38      0.21      0.27       115
    no_tumor       0.37      0.90      0.53       105
   pituitary       0.61      0.47      0.53        74

    accuracy                           0.42       394
   macro avg       0.49      0.43      0.39       394
weighted avg       0.48      0.42      0.37       394

13/13 - 1s - 104ms/step - accuracy: 0.4213 - loss: 13.5752
Accuracy:  0.42131978273391724
Loss:  13.57524585723877
